# Import libraries

In [1]:
#This notebook was run in the conda environment named "Jacob_Env1" (provided on GitHub)

In [2]:
#Load the required packages
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
from IPython.display import clear_output

import os
import sys
import numpy as np
import pandas as pd
import seaborn as sbn
import scanpy as sc
import csv
import anndata
import PIL
from PIL import Image
import multiprocessing
from multiprocessing.pool import ThreadPool
from collections import Counter

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import LinearSegmentedColormap

import warnings
warnings.filterwarnings("ignore")

# Define custom functions

In [3]:
#Define function to generate a plot with multiple subplots
def initialize_subplots(groups_to_plot, ncols = 3, figsize_multiplier = (7,5), gridspec_kw = None, figsize = None, print_help = True, **fig_kw):
    if type(groups_to_plot)==list:
        total = len(groups_to_plot)
    else:
        total = groups_to_plot
    nrows = int(np.ceil(total/ncols))
    if not figsize:
        figsize = (figsize_multiplier[0]*ncols, figsize_multiplier[1]*nrows)
    fig, axes = plt.subplots(nrows = nrows, ncols = ncols, figsize = figsize, gridspec_kw = gridspec_kw, **fig_kw)
    if print_help:
        if nrows>1 and ncols>1:
            print('ax = axes[ix // ncols, ix % ncols]')
        else:
            print('ax = axes[ix]')
    return fig, axes

In [4]:
#Define function to remove axis elements from subplots
#axes can either be an individual subplot, a list of individual subplots or mix of multiple subplots in a nested list 
def clean_axis(axes, remove_borders = True, remove_ticks = True, remove_axis_labels = True):
    axes = flatten_list(axes)
    print(axes)
    for ax in axes:
        if remove_borders:
            for loc in ['left','right','top','bottom']:
                ax.spines[loc].set_visible(False)
        if remove_ticks:
            for side in ['x','y']:
                ax.tick_params(
                    axis=side,  
                    which='both',
                    bottom=False, 
                    top=False, 
                    left=False,
                    labelbottom=False,
                    labelleft=False,
                    labeltop = False)
        if remove_axis_labels:
            ax.set_ylabel('')
            ax.set_xlabel('')

In [5]:
#Define function to make a flat, iteratable list out of a single element or a nested list
def flatten_list(lst):
    if type(lst)==np.ndarray:
        lst = lst.tolist()
    if type(lst)!=list:
        lst = [lst]
    elif len(lst)==1:
        lst = lst[0]
    elif not all([type(l)==list for l in lst]):
        lst = [[l] if type(l)!=list else l for l in lst]
        lst = [l for sublist in lst for l in sublist]
    else:
        lst = [l for sublist in lst for l in sublist]        
    return lst

In [6]:
#Define function to generate a tailor-made violinplot for online tool
def violinplot(gene, path = None, cells = None,title=None):
    mpl.rcParams['axes.linewidth']=0.25
    mult = 0.39      
    fig = plt.figure(constrained_layout=True, figsize = (14*mult,10*mult))
    fig.patch.set_facecolor('white')

    gs = GridSpec(4, 1, figure=fig, height_ratios = [2.5,2.5,2.5,2.5])
    ax0 = fig.add_subplot(gs[0, :])
    ax1 = fig.add_subplot(gs[1, :])
    ax2 = fig.add_subplot(gs[2, :])
    ax3 = fig.add_subplot(gs[3, :])

    df = adata.obs[[row, col]].copy()
    df[gene] = adata[:, gene].layers['normalized'].flatten()
    cmap = {cl: adata.uns[col+'_colors'][ix] for ix, cl in enumerate(adata.obs[col].cat.categories)}

    sbn.violinplot(data = df[df[row]=='E12.5'], x = col, y = gene, order=cluster_order, palette = cmap, ax = ax0, inner = None, cut = 0, scale = 'width',linewidth = 0.5)
    ax0.set_ylabel('E12.5\ncells',fontsize=8)
    ax0.set_xlabel('')
    sbn.violinplot(data = df[df[row]=='E13.5'], x = col, y = gene, order=cluster_order, palette = cmap, ax = ax1, inner = None, cut = 0, scale = 'width',linewidth = 0.5)
    ax1.set_ylabel('E13.5\ncells',fontsize=8)
    ax1.set_xlabel('')
    sbn.violinplot(data = df[df[row]=='E14.5'], x = col, y = gene, order=cluster_order, palette = cmap, ax = ax2, inner = None, cut = 0, scale = 'width',linewidth = 0.5)
    ax2.set_ylabel('E14.5\ncells',fontsize=8)
    ax2.set_xlabel('')
    sbn.violinplot(data = df, x = col, y = gene, order=cluster_order, palette = cmap, ax = ax3, inner = None, cut = 0, scale = 'width',linewidth = 0.5)
    ax3.set_ylabel('All\ncells',fontsize=8)
    ax3.set_xlabel('')
    ax3.set_xticklabels(ax3.get_xticklabels(), rotation=45,fontsize=8,ha = 'right')

    for ax_ in [ax0, ax1, ax2]:
        ax_.set_xticklabels([])
        ax_.set_xlabel('')
    for ax_ in [ax0, ax1, ax2, ax3]:
        ax_.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
        ax_.yaxis.set_major_locator(MaxNLocator(nbins = 2, integer = False))

        [ax_.axvline(border+0.5, ls = ':', c = 'lightgray') for border in group_borders]

    fig.suptitle(title,ha='center',weight='bold',fontsize='xx-large')
    ax0.set_title(gene,weight='semibold')
    
    #dpi can be adjusted based on the required quality of the saved figure
    fig.savefig(os.path.join(path, '{}_Plot1_{}.png'.format(cells, gene)), transparent = False, bbox_inches = 'tight', dpi = 100)  
    plt.close(fig)

In [7]:
#Define function to generate a tailor-made dotplot for online tool
def dotplot(gene, path = None, cells = None,x0=None,x1=None):
    sc.settings.verbosity = 0
    
    plots = ['E12.5','E13.5','E14.5','All']
    ncols = 1
    height_multiplier = 1.2 #Main paramter to control the y-spacing between the dots
    mult=0.39
    fig, axes = initialize_subplots(len(plots), ncols = ncols, sharex = True,figsize = (14*mult,7.5*mult), gridspec_kw = {'hspace':-0.9}, figsize_multiplier = (5,height_multiplier), print_help = False)
    fig.patch.set_facecolor('white')

    #Get the maximum colorvalue of all timepoints combined (to make colors comparable within the dotplot)
    df = adata.obs[[col]].copy()
    df[gene] = adata[:, gene].X
    #Dot color corresponds to the mean of the group (cluster)
    vmax = df.groupby(col).mean().max()[0]
    vmin = 0 if vmax > 0 else df.groupby(col).mean().min()[0]
    
    #Iterate over each timepoint
    for ix, t in enumerate(plots):
        ax = axes[ix]

        #Make a tempotary anndata object with cells just from the selected timepoint
        if t == 'All':
            tmp = adata[:, gene].copy()
        else:
            tmp = adata[adata.obs[row]==t][:, gene].copy()

            #If any cell population is missing from the timepoint, then fill in with empty gene expression
            if len(tmp.obs[col].cat.categories) != len(adata.obs[col].cat.categories):
                #get missing category names
                missing = [x for x in adata.obs[col].cat.categories if x not in tmp.obs[col].value_counts().index]
                #Make a dataframe with "NaN" expression for genes
                df = pd.DataFrame([np.nan]*len(missing), columns = [gene])
                #Convert to anndata object with missing clusters in obs
                placeholder = anndata.AnnData(df, obs = pd.DataFrame(missing, columns = [col]))
                #concatenate the placeholder anndata to the existing tmp object
                tmp = sc.concat([tmp, placeholder], join = 'outer')
        
        sc.settings.verbosity = 0
        #To ensure order of cell populations
        tmp.obs[row] = tmp.obs[row].astype('category')
        tmp.obs[col] = tmp.obs[col].astype('category')
        tmp.obs[col] = tmp.obs[col].cat.reorder_categories(cluster_order)

        #Make the dotplot using the scanpy DotPlot class (provides more options to adjust the plot)
        dp = sc.pl.DotPlot(tmp, var_names = gene, groupby = col, vmin = vmin, vmax = vmax, ax = ax)

        dp = dp.style(
            cmap='Blues',
            dot_max=1, #Have consistent values for dot with highest expression
            dot_min=0, #Have consistent values for dot with lowest expression
            x_padding=1, #Extra space added for the x-axis to make it look nicer
            y_padding=0,
            dot_edge_lw=0.1,
            )

        dp.legend(show = False)
        #Swap axes to have clusters on the x-axis
        dp.swap_axes()       
        #Get the actual dotplot axis
        ax_ = dp.get_axes()['mainplot_ax']

        #Remove tickmarks from all but last
        if ix != len(plots)-1:
            ax_.set_xticklabels([])
            ax_.set_xticks([])
        else:
            ax_.set_xticklabels([x.get_text().split(' [')[0] for x in ax_.get_xticklabels()], rotation = 45, ha = 'right',fontsize=8)

        if ix == 0:
            ax_.spines['bottom'].set_visible(False)
            ax_.spines['bottom'].set_linewidth(0.25)
            #Optional: Add gene name as the title of first plot
            #ax_.set_title(gene)
        elif ix == len(plots)-1:
            ax_.spines['top'].set_visible(False)
            ax_.spines['top'].set_linewidth(0.25)
        else:
            [ax_.spines[spine].set_visible(False) for spine in ['top', 'bottom']]
            [ax_.spines[spine].set_linewidth(0.25) for spine in ['top', 'bottom']]

        #Set embryonic age as yticklabel
        ax_.set_yticklabels([(t+' cells') if t!='All' else 'All cells'])

        #Add divider lines between groups of cell populations
        [ax_.axvline(border+1, ls = ':', c = 'lightgray') for border in group_borders]

        plt.gca()
        plt.subplots_adjust(top = 1.6, bottom = 0, right = 1, left = 0)
        plt.margins(0,0)
        clear_output()

    #Remove white space around the actual plot
    fig.canvas.draw()

    #Adjust the area for saving (manual optimization of x0, x1, y0, y1, so nothing is cut off)
    x0 = x0
    x1 = x1
    y0,y1 = -0.3, (len(plots)*height_multiplier/10)+0.03
    bbox = mpl.transforms.Bbox([[x0,y0],[x1,y1]])
    bbox = bbox.transformed(ax.transData).transformed(fig.dpi_scale_trans.inverted())

    #dpi can be adjusted based on the required quality of the saved figure
    fig.savefig(os.path.join(path, '{}_Plot2_{}.png'.format(cells, gene)), transparent = False, bbox_inches = bbox, pad_inches = None, dpi = 100)
    plt.close(fig)

In [8]:
#Define function to generate a tailor-made umapplot for online tool
def umapplot(gene, path = None, cells = None):
    mpl.rcParams['axes.linewidth']=0.25
    mult = 0.39      
    fig = plt.figure(constrained_layout=True, figsize = (14*mult,4*mult))
    fig.patch.set_facecolor('white')

    gs = GridSpec(1, 4, figure=fig)
    ax4 = fig.add_subplot(gs[-1, 0])
    ax5 = fig.add_subplot(gs[-1, 1])
    ax6 = fig.add_subplot(gs[-1, 2])
    ax7 = fig.add_subplot(gs[-1, 3])

    vmax = float(adata[:, gene].layers['normalized'].max())
    maxcount = round(float(adata[:, gene].layers['raw'].max()))

    for ix, E in enumerate(adata.obs[row].cat.categories):
        ax_ = [ax4, ax5, ax6][ix]
        sc.pl.umap(adata, ax = ax_, show = False, frameon = True, s = 10, title = '', alpha = 0.6)
        sc.pl.umap(adata[adata.obs[row]==E], color =gene, vmax=vmax, 
                layer='normalized', ax = ax_, show = False, 
                frameon = True, s = 10, title = '', alpha = 1, color_map=plt.cm.viridis)
        ax_.set_title(f'\n {E} cells only \n',fontsize=8)
        ax_.collections[-1].colorbar.remove() 

    vmax = np.ceil(vmax)
    sc.pl.umap(adata, color =gene, vmax=vmax, layer='normalized', ax = ax7, 
               show = False, frameon = True, s = 10, title = '', alpha = 1, color_map=plt.cm.viridis)
    ax7.set_title('All cells\n',fontsize=8)

    ax_fontsize = 7
    ax4.set_xlabel(ax4.get_xlabel(), fontsize = ax_fontsize, loc = 'left')
    ax4.set_ylabel(ax4.get_ylabel(), fontsize = ax_fontsize, loc = 'bottom')

    [ax_.set_xlabel('') for ax_ in [ax5, ax6, ax7]]
    [ax_.set_ylabel('') for ax_ in [ax5, ax6, ax7]]
    ax7.set_xlabel(f'[Max. mRNA copies\nper cell: {maxcount}]',fontsize = 8)

    cbar = ax7.collections[0].colorbar
    cbar.ax.yaxis.set_major_locator(MaxNLocator(nbins = 2, integer = True, min_n_ticks = 3))
    cbar.ax.tick_params(labelsize=6)

    #dpi can be adjusted based on the required quality of the saved figure
    fig.savefig(os.path.join(path, '{}_Plot3_{}.png'.format(cells, gene)), transparent = False, bbox_inches = 'tight', dpi = 100)  
    plt.close(fig)

In [9]:
#Define function that concatenates all 3 plots for each gene (violinplot, dotplot, umapplot) vertically
def plotconcatenate(gene, path = None, cells = None):
    #Read in all plots that were generated for a particular gene (Plot1, Plot2, Plot3 in our case) in the folder of a particular cell type (All, FIB, or EPI)
    list_im = [os.path.join(path, '{}_Plot{}_{}.png'.format(cells, i, gene)) for i in range(1,4)]
    imgs    = [ Image.open(i) for i in list_im ]
    #Pick the image which is the smallest, and resize the others to match it (can be arbitrary image shape here)
    min_shape = sorted([i.size[0] for i in imgs])[0]
    imgs_comb = np.vstack([i.resize((min_shape,i.size[1])) for i in imgs])
    #Combine into one image
    imgs_comb = Image.fromarray( imgs_comb)
    imgs_comb.save(os.path.join(path, 'Combined', '{}_Assembled-Plots_{}.png'.format(cells, gene)), transparent = False)

# Universal settings

In [10]:
#Set path
path = '/mnt/c/Users/User/Jacob-Et-Al'

#Required folder structure:
#One output folders named "Tool-Figures"
#Under "Tool-Figures", sub-folders named "All","FIB","EPI","Combined"
#In 3 of the sub-folders (All, FIB, EPI) generate another sub-folder named "Combined"

In [11]:
#Define color map
colors = [(0.8, 0.8, 0.8), (0.19, 0.04, 0.55)] # first color is grey, last is dark purple
cm = LinearSegmentedColormap.from_list("Custom", colors, N=200)

In [12]:
#Define the metadata item to divide by (in our case we only used "embryonic age")
row = 'embryonic_age'

# Major cell types

## Import data and adjust settings

In [13]:
#Define which cells will be plotted (in this section: Major Cell Types)
cells='All'

In [14]:
#Load h5 dataset with all required information (clustering, umap coordinates, expression values, etc.)
adata = sc.read_h5ad(os.path.join(path, 'Notebook-Output','Adata-object_All.h5'))

In [15]:
#Define which clustering to use and how to order cell populations in plots (from left to right)
col = 'HC_named'
cluster_order = ['FIB','EPI','MUSCLE Early','MUSCLE Mid','MUSCLE Late','VESSEL BECs','VESSEL LECs','VESSEL MuralCells','IMMU DendriticCells','IMMU Macrophages','IMMU MastCells','NC Melanocytes','NC SchwannCells']
#Define between which clusters to draw dotted lines (for easier navigation within the plot)
group_borders = [0,1,4,7,10] 
#Define dimensions of dotplot bbox (required some playing around to find ideal values)
x0=-0.135
x1=0.975
#Define title for plots
title='All Cell Types (First-Level-Clustering)'

## Define functions (that need to be re-defined for each cell type)

In [16]:
#Define function that generates all 3 plots for each gene (violinplot, dotplot, umapplot)
def plotting_wrapper(gene,cells=cells):
    global ix
    global path
    print(gene, flush = True)
    print(ix+1, flush = True)
    ix+=1
    clear_output(wait = True)
    outpath = os.path.join(path, 'Tool-Figures', cells)
    try:
        violinplot(gene, path = outpath, cells = cells,title=title)
        dotplot(gene, path = outpath, cells = cells,x0=x0,x1=x1)
        umapplot(gene, path = outpath, cells = cells)
    except:
        print(f'Error: {gene}', flush = True)

In [17]:
#Small function that helps running plotconcatenate with multicoreprocessing
def concatenation(gene, path = os.path.join(path, 'Tool-Figures', cells), cells=cells):
    plotconcatenate(gene, path = path, cells = cells)

## Generate individual plots

In [ ]:
#Generate plots for all genes with the help of the plotting_wrapper function
#Number of cores can be adjusted based on computing power
chunk = 1000
ix = 0
for i in range(0, len(adata.var_names), chunk):
    print(i)
    pool = multiprocessing.Pool(processes=5)
    pool.imap(plotting_wrapper, adata.var_names[i:i+chunk])
    pool.close()
    pool.join()
print('Done')

In [ ]:
#Check which/how many genes have already been plotted
filelist=os.listdir(os.path.join(path, 'Tool-Figures', cells))
completed_genes=list([x.split('_')[-1].split('.pn')[0] for x in filelist if x != 'Combined']) 
genes = [g for g, v in Counter(completed_genes).items() if v < 3]
for g in genes:
    print(f'{g}: {Counter(completed_genes)[g]}')

In [ ]:
#Rerun plotting_wrapper for genes not yet completed
chunk = 1000
ix = 0
genes = [g for g in adata.var_names if any([g not in completed_genes, g in genes])]
for i in range(0, len(genes), chunk):
    print(i)
    pool = multiprocessing.Pool(processes=5)
    pool.imap(plotting_wrapper, genes[i:i+chunk])
    pool.close()
    pool.join()
print('Done')

In [ ]:
#Check once more which/how many genes have already been plotted
filelist = os.listdir(os.path.join(path, 'Tool-Figures', cells))
completed_genes = list([x.split('_')[-1].split('.pn')[0] for x in filelist if x != 'Combined'])
genes = [g for g, v in Counter(completed_genes).items() if v < 3]
for g in genes:
    print(f'{g}: {Counter(completed_genes)[g]}')

In [ ]:
#Show the number of completed genes
len(completed_genes)/3

In [ ]:
#Show the content of the anndata object to see how many genes there are (19994 in our case)
adata

## Combine 3 plots into 1

In [18]:
#Concatenate violinplot, dotplot, and umapplot. Do so for each gene.
pool = ThreadPool(500)
pool.imap_unordered(concatenation, adata.var_names)
pool.close()

# Fibroblasts

## Import data and adjust settings

In [ ]:
#Define which cells will be plotted (in this section: Fibroblasts)
cells='FIB'

In [ ]:
#Load h5 dataset with all required information (clustering, umap coordinates, expression values, etc.)
adata = sc.read_h5ad(os.path.join(path, 'Notebook-Output','Adata-object_FIB.h5'))

In [ ]:
#Define which clustering to use and how to order cell populations in plots (from left to right)
col = 'leiden_named_DCsub'
cluster_order = ['FIB Origin1','FIB Origin2','FIB Origin3','FIB Origin4','FIB Origin5','FIB Origin6','FIB Deep1','FIB Deep2','FIB Deep3','FIB Inter1','FIB Inter2','FIB Inter3','FIB Muscle1','FIB Muscle2','FIB Lower','FIB Upper1','FIB Upper2','FIB Upper3','FIB Upper4','FIB EarlyDC','FIB LateDC','CHOND']
#Define between which clusters to draw dotted lines (for easier navigation within the plot)
group_borders = [5, 8,11,13,14,18,20] 
#Define dimensions of dotplot bbox (required some playing around to find ideal values)
x0=-0.135
x1=0.975
#Define title for plots
title='Fibroblast Subpopulations'

## Define functions (that need to be re-defined for each cell type)

In [ ]:
#Define function that generates all 3 plots for each gene (violinplot, dotplot, umapplot)
def plotting_wrapper(gene,cells=cells):
    global ix
    global path
    print(gene, flush = True)
    print(ix+1, flush = True)
    ix+=1
    clear_output(wait = True)
    outpath = os.path.join(path, 'Tool-Figures', cells)
    try:
        violinplot(gene, path = outpath, cells = cells,title=title)
        dotplot(gene, path = outpath, cells = cells,x0=x0,x1=x1)
        umapplot(gene, path = outpath, cells = cells)
    except:
        print(f'Error: {gene}', flush = True)

In [ ]:
#Small function that helps running plotconcatenate with multicoreprocessing (define for each cell-type)
def concatenation(gene, path = os.path.join(path, 'Tool-Figures', cells), cells=cells):
    plotconcatenate(gene, path = path, cells = cells)

## Generate individual plots

In [ ]:
#Generate plots for all genes with the help of the plotting_wrapper function
#Number of cores can be adjusted based on computing power
chunk = 1000
ix = 0
for i in range(0, len(adata.var_names), chunk):
    print(i)
    pool = multiprocessing.Pool(processes=5)
    pool.imap(plotting_wrapper, adata.var_names[i:i+chunk])
    pool.close()
    pool.join()
print('Done')

In [ ]:
#Check which/how many genes have already been plotted
filelist=os.listdir(os.path.join(path, 'Tool-Figures', cells))
completed_genes=list([x.split('_')[-1].split('.pn')[0] for x in filelist if x != 'Combined']) 
genes = [g for g, v in Counter(completed_genes).items() if v < 3]
for g in genes:
    print(f'{g}: {Counter(completed_genes)[g]}')

In [ ]:
#Rerun plotting_wrapper for genes not yet completed
chunk = 1000
ix = 0
genes = [g for g in adata.var_names if any([g not in completed_genes, g in genes])]
for i in range(0, len(genes), chunk):
    print(i)
    pool = multiprocessing.Pool(processes=5)
    pool.imap(plotting_wrapper, genes[i:i+chunk])
    pool.close()
    pool.join()
print('Done')

In [ ]:
#Check once more which/how many genes have already been plotted
filelist = os.listdir(os.path.join(path, 'Tool-Figures', cells))
completed_genes = list([x.split('_')[-1].split('.pn')[0] for x in filelist if x != 'Combined'])
genes = [g for g, v in Counter(completed_genes).items() if v < 3]
for g in genes:
    print(f'{g}: {Counter(completed_genes)[g]}')

In [ ]:
#Show the number of completed genes
len(completed_genes)/3

In [ ]:
#Show the content of the anndata object to see how many genes there are (19994 in our case)
adata

## Combine 3 plots into 1

In [ ]:
#Concatenate violinplot, dotplot, and umapplot. Do so for each gene.
pool = ThreadPool(500)
pool.imap_unordered(concatenation, adata.var_names)
pool.close()

# Epidermis

## Import data and adjust settings

In [ ]:
#Define which cells will be plotted (in this section: Epidermis)
cells='EPI'

In [ ]:
#Load h5 dataset with all required information (clustering, umap coordinates, expression values, etc.)
adata = sc.read_h5ad(os.path.join(path, 'Notebook-Output','Adata-object_EPI.h5'))

In [ ]:
#Define which clustering to use and how to order cell populations in plots (from left to right)
col = 'HC_named'
cluster_order = [ 'EPI BasalTagln','EPI Basal1','EPI Basal2','EPI Basal3','EPI Basal4', 'EPI EarlyPlacode', 'EPI LatePlacode', 'EPI EarlyDiff', 'EPI LateDiff', 'EPI Periderm']
#Define between which clusters to draw dotted lines (for easier navigation within the plot)
group_borders = [4, 6] 
#Define dimensions of dotplot bbox (required some playing around to find ideal values)
x0=-0.13
x1=0.935
#Define title for plots
title='Epidermal Subpopulations'

## Define functions (that need to be re-defined for each cell type)

In [ ]:
#Define function that generates all 3 plots for each gene (violinplot, dotplot, umapplot)
def plotting_wrapper(gene,cells=cells):
    global ix
    global path
    print(gene, flush = True)
    print(ix+1, flush = True)
    ix+=1
    clear_output(wait = True)
    outpath = os.path.join(path, 'Tool-Figures', cells)
    try:
        violinplot(gene, path = outpath, cells = cells,title=title)
        dotplot(gene, path = outpath, cells = cells,x0=x0,x1=x1)
        umapplot(gene, path = outpath, cells = cells)
    except:
        print(f'Error: {gene}', flush = True)

In [ ]:
#Small function that helps running plotconcatenate with multicoreprocessing (define for each cell-type)
def concatenation(gene, path = os.path.join(path, 'Tool-Figures', cells), cells=cells):
    plotconcatenate(gene, path = path, cells = cells)

## Generate individual plots

In [ ]:
#Generate plots for all genes with the help of the plotting_wrapper function
#Number of cores can be adjusted based on computing power
chunk = 1000
ix = 0
for i in range(0, len(adata.var_names), chunk):
    print(i)
    pool = multiprocessing.Pool(processes=5)
    pool.imap(plotting_wrapper, adata.var_names[i:i+chunk])
    pool.close()
    pool.join()
print('Done')

In [ ]:
#Check which/how many genes have already been plotted
filelist=os.listdir(os.path.join(path, 'Tool-Figures', cells))
completed_genes=list([x.split('_')[-1].split('.pn')[0] for x in filelist if x != 'Combined']) 
genes = [g for g, v in Counter(completed_genes).items() if v < 3]
for g in genes:
    print(f'{g}: {Counter(completed_genes)[g]}')

In [ ]:
#Rerun plotting_wrapper for genes not yet completed
chunk = 1000
ix = 0
genes = [g for g in adata.var_names if any([g not in completed_genes, g in genes])]
for i in range(0, len(genes), chunk):
    print(i)
    pool = multiprocessing.Pool(processes=5)
    pool.imap(plotting_wrapper, genes[i:i+chunk])
    pool.close()
    pool.join()
print('Done')

In [ ]:
#Check once more which/how many genes have already been plotted
filelist = os.listdir(os.path.join(path, 'Tool-Figures', cells))
completed_genes = list([x.split('_')[-1].split('.pn')[0] for x in filelist if x != 'Combined']) 
genes = [g for g, v in Counter(completed_genes).items() if v < 3]
for g in genes:
    print(f'{g}: {Counter(completed_genes)[g]}')

In [ ]:
#Show the number of completed genes
len(completed_genes)/3

In [ ]:
#Show the content of the anndata object to see how many genes there are (19994 in our case)
adata

## Combine 3 plots into 1

In [ ]:
#Concatenate violinplot, dotplot, and umapplot. Do so for each gene.
pool = ThreadPool(500)
pool.imap_unordered(concatenation, adata.var_names)
pool.close()

# Assemble combined Major Cell Types, FIB, and EPI plots into one item

In [ ]:
#Generate a white spaceholder to have a bit more room between the 3 assembled items
fig, ax = plt.subplots(figsize = (71.7, 8))
fig.patch.set_facecolor('white')
clean_axis(ax)
fig.savefig(os.path.join(path, 'Tool-Figures', 'Spaceholder.png'), transparent = False, bbox_inches = 'tight', dpi = 10)

In [ ]:
#Define function that concatenates all 3 combined items for each gene (Major Cell Types, FIB, and EPI) vertically
def final_plotconcatenate(gene, path = None):
    #From "Combined" subfolder in "All", "FIB", "EPI" folder read in assembled item for gene of interest
    list_im = [os.path.join(path, cell_type, 'Combined', '{}_Assembled-Plots_{}.png'.format(cell_type, gene)) for cell_type in ['All','FIB','EPI']]+[os.path.join(path, 'Spaceholder.png')]
    #Define order in which items should be concatenated (this is also where the spaceholder is added)
    list_im = [list_im[i] for i in [0,3,1,3,2]]
    imgs    = [ Image.open(i) for i in list_im ]
    #Pick the image which is the smallest, and resize the others to match it (can be arbitrary image shape here)
    min_shape = sorted([i.size[0] for i in imgs])[0] #Maybe something gets messed up here? While resizing the images? I guess this is quite possible for the sqeezing in y. But then we also need to move the dotplot right so it is right aligned with the violin plot
    imgs_comb = np.vstack([i.resize((min_shape,i.size[1])) for i in imgs])
    #Combine into one image
    imgs_comb = Image.fromarray( imgs_comb)
    imgs_comb.save(os.path.join(path,'Combined', '{}.png'.format(gene)), transparent = False)

In [ ]:
#Small function that helps running final_plotconcatenate with multicoreprocessing
def final_concatenation(gene, path = os.path.join(path, 'Tool-Figures')):
    final_plotconcatenate(gene, path = path)

In [ ]:
#Concatenate combined plots (Major Cell Types, FIB, EPI) into one. Do so for each gene.
pool = ThreadPool(500)
pool.imap_unordered(final_concatenation, adata.var_names)
pool.close()

In [ ]:
#Check which/how many genes have already been plotted
filelist = os.listdir(os.path.join(path, 'Tool-Figures', 'Combined'))
completed_genes = list([x.split('.pn')[0] for x in filelist]) 

In [ ]:
#Show the number of completed genes
print(len(completed_genes))